In [1]:
from func import getMeanStd
from pickle import *
import torch
import torchvision
import json
from torchvision import transforms
import sys
sys.path.insert(0,"/home/students/cnn/Deep-Learnng")

from Detection.Loader import*
from Detection.CocoDataset import labelboxCoco
from Detection.util_labelbox import *
from Detection.util_detection import*
from Detection.util_train import*
from Detection.util_filter import*
from Detection.util_geolocation import *
import csv 
import shutil
import os

In [18]:
base_model_path="/home/students/cnn/Deep-Learnng/base_model.pt"
model_path="/home/students/cnn/Deep-Learnng/Model_Experiment/models/model.pt"

batch_size=6
newSize=(1000,1000)
epochs=5

In [35]:
#Using GPU or CPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.backends.cuda.cufft_plan_cache.clear()
else:
    device = torch.device('cpu')

transform = transforms.Compose([
                transforms.Resize(newSize),
                transforms.ToTensor()])
                #transforms.Normalize([0.3932, 0.3908, 0.3942],[0.2291, 0.2268, 0.2334])])

In [40]:
ucf=labelboxCoco("/home/students/cnn/all",["../Data/UCFcoco.json","../Data/CMPcoco.json","../Data/DOORKNOBcoco.json"],newSize,transform=transform)
loader_train=Loader(ucf,start=100,batch_size=batch_size,shuffle=True)
print(len(loader_train))
loader_ucf=Loader(ucf,end=100,batch_size=batch_size)
loader=Loader(ucf,start=200,end=300,batch_size=batch_size)


nyc=labelboxCoco("/home/students/cnn/NYC",["../Data/NYCcoco.json"],newSize,transform=transform)
loader_nyc=Loader(nyc,end=100,batch_size=batch_size)
# loader_val=Loader(labelbox,end=100,batch_size=batch_size)
# print(len(loader_val))

TypeError: expected str, bytes or os.PathLike object, not list

In [27]:
cmp=labelboxCoco("/home/students/cnn/CMP","../Data/CMPcoco.json",newSize,transform=transform)
loader_cmp=Loader(cmp,batch_size=batch_size,shuffle=True)

door=labelboxCoco("/home/students/cnn/DOORKNOB","../Data/DOORKNOBcoco.json",newSize,transform=transform)
loader_door=Loader(door,batch_size=batch_size,shuffle=True)

In [21]:
#Set up the model
model=torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

#4 categories + 1 background
model=transfer(model,5)
#model=torch.load(base_model_path,map_location=device)
# print(model)
for param in model.parameters():
    param.requires_grad = True
# for param in model.rpn.parameters():
#     param.requires_grad = True
# for param in model.roi_heads.parameters():
#     param.requires_grad = True


# for name, param in model.named_parameters():
#     print(name, param.requires_grad)
optimizer=torch.optim.Adam(model.parameters(),lr=1e-4,betas=(0.9, 0.95))
model=train(model,optimizer,epochs,
              loader_train=loader_train,loader_val=[loader_ucf,loader,loader_nyc],device=device,wb=False)



  0%|          | 0/924 [00:00<?, ?it/s]

Device: cuda


 99%|█████████▉| 918/924 [02:03<00:00,  7.46it/s]


*************************Recall Precision **************************************
Door -> TP: 98  Predict: 1347  Truth: 100  Precision:7.28%  Recall:98.00%
Knob -> TP: 16  Predict: 488  Truth: 48  Precision:3.28%  Recall:33.33%
Stairs -> TP: 7  Predict: 1114  Truth: 12  Precision:0.63%  Recall:58.33%
Ramp -> TP: 0  Predict: 16  Truth: 0  Precision:0.00%  Recall:0.00%
*******************************************************************************
*************************Recall Precision **************************************
Door -> TP: 98  Predict: 1169  Truth: 100  Precision:8.38%  Recall:98.00%
Knob -> TP: 13  Predict: 505  Truth: 28  Precision:2.57%  Recall:46.43%
Stairs -> TP: 33  Predict: 1183  Truth: 41  Precision:2.79%  Recall:80.49%
Ramp -> TP: 2  Predict: 36  Truth: 18  Precision:5.56%  Recall:11.11%
*******************************************************************************


  0%|          | 0/924 [00:00<?, ?it/s]

...

*************************Recall Precision **************************************
Door -> TP: 89  Predict: 380  Truth: 100  Precision:23.42%  Recall:89.00%
Knob -> TP: 27  Predict: 179  Truth: 48  Precision:15.08%  Recall:56.25%
Stairs -> TP: 8  Predict: 61  Truth: 12  Precision:13.11%  Recall:66.67%
Ramp -> TP: 0  Predict: 89  Truth: 0  Precision:0.00%  Recall:0.00%
*******************************************************************************
*************************Recall Precision **************************************
Door -> TP: 98  Predict: 393  Truth: 100  Precision:24.94%  Recall:98.00%
Knob -> TP: 23  Predict: 167  Truth: 28  Precision:13.77%  Recall:82.14%
Stairs -> TP: 40  Predict: 107  Truth: 41  Precision:37.38%  Recall:97.56%
Ramp -> TP: 11  Predict: 75  Truth: 18  Precision:14.67%  Recall:61.11%
*******************************************************************************


  0%|          | 0/924 [00:00<?, ?it/s]

*************************Recall Precision **************************************
Door -> TP: 126  Predict: 318  Truth: 164  Precision:39.62%  Recall:76.83%
Knob -> TP: 33  Predict: 140  Truth: 77  Precision:23.57%  Recall:42.86%
Stairs -> TP: 48  Predict: 115  Truth: 96  Precision:41.74%  Recall:50.00%
Ramp -> TP: 2  Predict: 70  Truth: 10  Precision:2.86%  Recall:20.00%
*******************************************************************************
Epochs: 4  Time used: 123 s  loss_train: tensor(0.2044, device='cuda:0')  loss_val: tensor(0.4953, device='cuda:0') 



 99%|█████████▉| 918/924 [02:03<00:00,  7.42it/s]


*************************Recall Precision **************************************
Door -> TP: 89  Predict: 439  Truth: 100  Precision:20.27%  Recall:89.00%
Knob -> TP: 31  Predict: 294  Truth: 48  Precision:10.54%  Recall:64.58%
Stairs -> TP: 8  Predict: 120  Truth: 12  Precision:6.67%  Recall:66.67%
Ramp -> TP: 0  Predict: 58  Truth: 0  Precision:0.00%  Recall:0.00%
*******************************************************************************
*************************Recall Precision **************************************
Door -> TP: 100  Predict: 437  Truth: 100  Precision:22.88%  Recall:100.00%
Knob -> TP: 23  Predict: 284  Truth: 28  Precision:8.10%  Recall:82.14%
Stairs -> TP: 41  Predict: 144  Truth: 41  Precision:28.47%  Recall:100.00%
Ramp -> TP: 8  Predict: 65  Truth: 18  Precision:12.31%  Recall:44.44%
*******************************************************************************
*************************Recall Precision **************************************
Door -> TP: 11

In [22]:
torch.save(model,"/home/students/cnn/Deep-Learnng/Model_Experiment/models/base_model_mean.pt")

In [28]:
cmp_mean=getMeanStd(loader_cmp)

print("cmp:",cmp_mean)

cmp: (tensor([0.4785, 0.4532, 0.4160]), tensor([0.2385, 0.2301, 0.2298]))


In [31]:
door_mean=getMeanStd(loader_door)

print("door mean:",door_mean)

door mean: (tensor([0.5267, 0.4758, 0.4255]), tensor([0.2616, 0.2680, 0.2820]))


[1, 2, 3, 4]
